<a href="https://colab.research.google.com/github/Piontk/Oracle-Database-Inverted-File-Flat-IVF-indexes-/blob/main/IVF_parte3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-text-splitters==0.3.8 langchain==0.3.24 langchain-community==0.3.23 langchain-openai==0.3.33 langchain-oracledb==1.0.2 pypdf oracledb python-dotenv

In [2]:
from langchain_openai import OpenAIEmbeddings
import oracledb
import array
from dotenv import load_dotenv
import os

load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
CONNECT_STRING = os.getenv("CONNECT_STRING")
WALLET_PASSWORD = os.getenv("WALLET_PASSWORD")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

oracledb.defaults.fetch_lobs = False

In [3]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
pergunta = "How to grant a role locally?"
vector = embeddings.embed_query(pergunta)
vector_arr = array.array("f", vector)

In [6]:
accuracy_query = """
DECLARE
    q_v VECTOR;
    report VARCHAR2(128);
BEGIN
    q_v := :vec;
    report := dbms_vector.index_accuracy_query(
        OWNER_NAME => 'VECTOR',
        INDEX_NAME => 'IDX_IVF_ORACLE_DOCS_01',
        qv => q_v,
        top_k => 5,
        target_accuracy => 90
    );
    :out := report;
END;
"""


conn = oracledb.connect(user=DB_USER,
                        password=DB_PASSWORD,
                        dsn=CONNECT_STRING,
                        config_dir="/content/wallet",
                        wallet_location="/content/wallet",
                        wallet_password=WALLET_PASSWORD )

cursor = conn.cursor()
out_var = cursor.var(str)
cursor.execute(accuracy_query, vec=vector_arr, out=out_var)

print(out_var.getvalue())

Accuracy achieved (100%) is 10% higher than the Target Accuracy requested (90%)


In [14]:

accuracy_report = """
DECLARE
    rep_id NUMBER;
BEGIN
    rep_id := DBMS_VECTOR.INDEX_ACCURACY_REPORT(
        owner_name => 'VECTOR',
        index_name => 'IDX_IVF_ORACLE_DOCS_01',
        start_time => TO_TIMESTAMP_TZ('2026-01-07 20:00:00 -3:00', 'YYYY-MM-DD HH24:MI:SS TZH:TZM'),
        end_time => TO_TIMESTAMP_TZ('2026-01-07 21:00:00 -3:00', 'YYYY-MM-DD HH24:MI:SS TZH:TZM')
    );
    :out := rep_id;
END;
"""


conn = oracledb.connect(user=DB_USER,
                        password=DB_PASSWORD,
                        dsn=CONNECT_STRING,
                        config_dir="/content/wallet",
                        wallet_location="/content/wallet",
                        wallet_password=WALLET_PASSWORD )

cursor = conn.cursor()
out_var = cursor.var(str)
cursor.execute(accuracy_report, out=out_var)

print(out_var.getvalue())

6
